In [28]:
import numpy as np
import pandas as pd
import pickle
# SQL related packages
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
# sklearn packages
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics, preprocessing
from sklearn.linear_model import LogisticRegression as Log
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics.scorer import make_scorer
# text analysis packages
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec, Word2Vec

In [55]:
# access to sql database
dbname = 'patent_db'
username = 'jy'
pswd = 'jy'

engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

# reading from sql database
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [56]:
# read data from 2004-2007
years = np.arange(2004, 2008)

# dataframe to store the results
claims = pd.DataFrame()

# import the abstract from each table
for year in years:
    # query:
    sql_query = """
    SELECT claims, id, payment_times, classification
        FROM patents_%s;
    """ %str(year)

    results = pd.read_sql_query(sql_query,con)
    
    claims = pd.concat([claims, results], axis = 0)
    
# check size of the data
claims.shape

(12033, 4)

In [57]:
# extract the response variable
# reformat the response variable into binary
y_data = np.zeros(claims.shape[0])
y_data[claims['payment_times'].values >= 2] = 1

print "Percentage of patents with > 1 maintenance fee payments: ", np.mean(y_data)

Percentage of patents with > 1 maintenance fee payments:  0.628521565694


In [58]:
# remove the patents missing classification data
missing_class_index = (claims['classification'].values == 'NA')

# reassign patent index
claims.index = range(len(claims.index))
# drop the rows
claims =  claims.drop(claims.index[missing_class_index])
claims.shape

(12030, 4)

In [59]:
y_data = y_data[claims['classification'].values != 'NA']
y_data.shape

(12030,)

In [60]:
# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(claims, y_data[:claims.shape[0]], 
                                                    test_size = 2000, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (10030, 4)
x_test:  (2000, 4)
y_train:  (10030,)
y_test:  (2000,)


In [61]:
def tokenize_cleaning(text):
    # tokenize the text first
    tokens = word_tokenize(text.decode('utf-8'))
    
    # lowercase all the words
    tokens = [w.lower() for w in tokens]
    
    # clean up stop words and punctuations 
    stop_list = stopwords.words('english') + list(string.punctuation)

    tokens_no_stop = [token for token in tokens
                        if token not in stop_list]            
    
#     # extract stem of the words
#     stemmer = PorterStemmer()
#     tokens_stem = [stemmer.stem(token) for token in tokens_no_stop]

    # use lemma instead
    # reason: remove the influence of plural or tense
    # but retain the subtle difference in legal writting
    lemmatizer = WordNetLemmatizer()
    tokens_lemma = [lemmatizer.lemmatize(token) for token in tokens_no_stop]
    
    # remove numbers (the actual values are not useful)
    tokens_no_num = []
    for token in tokens_lemma:
        try:
            float(token)
        except:
            tokens_no_num.append(token)
    
    return tokens_no_num

In [63]:
# tokenize_clean the data for word2vec training

cleaned_text = []
for i in range(x_train.shape[0]):
    tokens = tokenize_cleaning(x_train['claims'].iloc[i])
    cleaned_text.append(tokens)

In [64]:
model = Word2Vec(cleaned_text, size=100, window=5, min_count=5, workers=4)

In [65]:
model.wv.most_similar(positive=['protein'])

[(u'polypeptide', 0.7180240154266357),
 (u'mutant', 0.6947092413902283),
 (u'subunit', 0.6833655834197998),
 (u'kinase', 0.6764980554580688),
 (u'receptor', 0.6745332479476929),
 (u'gfp', 0.669427752494812),
 (u'translocation', 0.6464871764183044),
 (u'chimeric', 0.6401758193969727),
 (u'pi3', 0.6384416222572327),
 (u'app', 0.6349186897277832)]

In [66]:
# save the word2vec model
model.save('models/word2vec_claims_2')

In [42]:
# load the word2vec model
model = Word2Vec.load('models/word2vec_claims_2')

In [67]:
# tokenize_clean for training the final model

cleaned_text = []
for i in range(claims.shape[0]):
    tokens = tokenize_cleaning(claims['claims'].iloc[i])
    cleaned_text.append(tokens)

In [68]:
# compute average word-vector for a text
def dec_vec(model, text):
    # store the vector for each word
    vectors = []
    
    # compute on each word
    for j in range(len(text)):
        try:
            vectors.append(model.wv[text[j]])
        except:
            continue
    
    if not vectors:
        vectors_mean = np.zeros((1, 100))
    else:
        vectors_mean = np.nanmean(vectors, axis = 0)
        vectors_mean = vectors_mean.reshape((1, 100))
    
    # return vector mean
    return vectors_mean

In [69]:
claims_vec = np.zeros((1, 100))

for i in range(claims.shape[0]):
    vec = dec_vec(model, cleaned_text[i])
    claims_vec = np.concatenate([claims_vec, vec], axis = 0)

In [70]:
claims_vec = claims_vec[1:, :]
claims_vec.shape

(12030, 100)

In [71]:
# split the training and testing data
# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(claims_vec, y_data, 
                                                    test_size = 2000, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (10030, 100)
x_test:  (2000, 100)
y_train:  (10030,)
y_test:  (2000,)


In [72]:
# sample the same number of'useful' patents as the 'not useful' patents
# size of each class
num_size = np.sum(y_data == 0)

#random shuffle the rows
n = x_train.shape[0]
perm = range(n)
np.random.shuffle(perm)

x_train = x_train[perm]
y_train = y_train[perm]

# separate the two classes
x_useful = x_train[y_train == 1, :]
x_not_useful = x_train[y_train == 0, :]
y_useful = y_train[y_train == 1]
y_not_useful = y_train[y_train == 0]

# sample num_size from the 'useful' class
x_useful = x_useful[:num_size]
y_useful = y_useful[:num_size]

# combine the two classes
x_data_sub = np.concatenate((x_useful, x_not_useful), axis = 0)
y_data_sub = np.concatenate((y_useful, y_not_useful), axis = 0)

# shuffle again
# shuffle the combined data
n2 = x_data_sub.shape[0]
perm2 = range(n2)
np.random.shuffle(perm2)

x_data_sub = x_data_sub[perm2]
y_data_sub = y_data_sub[perm2]

# check the size
print x_data_sub.shape
print y_data_sub.shape

(8217, 100)
(8217,)


In [73]:
# standardize the predictors
scaler = preprocessing.StandardScaler()

x_train_std = scaler.fit_transform(x_data_sub)
x_test_std = scaler.transform(x_test)

In [74]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 8, 2)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train_std, y_data_sub)

In [76]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.545332846538
Best parameter:  {'max_features': 4}


In [75]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_std, y_data_sub)

y_pred = best_rf.predict(x_test_std)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.5835
F1 score:  0.687664041995
Precision:  0.660662824207
Recall:  0.716966379984


array([[250, 471],
       [362, 917]])

In [77]:
# write my own scoring function
def my_loss_func(y_actual, y_pred):
    
    # set the price of applying for each patent
    patent_cost = 10
    
    # set the value of a useful patent
    useful_value = 15
    
    total_cost = np.sum(y_pred == 1) * patent_cost
    total_value = np.sum((y_actual == 1) & (y_pred == 1)) * useful_value
    
    profit = total_value -  total_cost
    
    return profit

my_scorer = make_scorer(my_loss_func, greater_is_better=True)

In [78]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 8, 2)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = my_scorer)
# fit on the data
grid_model = grid_model.fit(x_train_std, y_data_sub)

In [79]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: -1590.00669344
Best parameter:  {'max_features': 6}


In [80]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_std, y_data_sub)

y_pred = best_rf.predict(x_test_std)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.5805
F1 score:  0.687523277467
Precision:  0.656472261735
Recall:  0.721657544957
Net value:  -215


array([[238, 483],
       [356, 923]])

In [83]:
# tune class weight of random forest
model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 8, 2)

# add class weight tuning to the random forest model
weights = np.logspace(-3,3,7)
weight_list_dict = [{0:1, 1: weights[i]} for i in range(len(weights))]

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space,
                                       'class_weight': weight_list_dict}, 
                          cv  = 5, scoring = my_scorer)
# fit on the data
grid_model = grid_model.fit(x_train_std, y_data_sub)

In [84]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: -1243.99537544
Best parameter:  {'max_features': 2, 'class_weight': {0: 1, 1: 1000.0}}


In [85]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_std, y_data_sub)

y_pred = best_rf.predict(x_test_std)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.535
F1 score:  0.610878661088
Precision:  0.657065706571
Recall:  0.570758405004
Net value:  -160


array([[340, 381],
       [549, 730]])

In [81]:
# if everything is predicted to be useful (benchmark)
y_pred_uniform = np.ones(y_pred.shape)
# accuracy
print "Test accuracy: ", np.mean(y_pred_uniform == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred_uniform)
print "Precision: ", metrics.precision_score(y_test, y_pred_uniform)
print "Recall: ", metrics.recall_score(y_test, y_pred_uniform)
print "Net value: ", my_loss_func(y_test, y_pred_uniform)
metrics.confusion_matrix(y_test, y_pred_uniform)

Test accuracy:  0.6395
F1 score:  0.780115888991
Precision:  0.6395
Recall:  1.0
Net value:  -815


array([[   0,  721],
       [   0, 1279]])

In [82]:
# if randomly assign the two classes with the same ratio
y_pred_random = np.random.uniform(0, 1, size = y_test.shape[0])
# turn into binary
y_random_class = np.zeros(y_test.shape[0]) 
y_random_class[y_pred_random > 1- np.mean(y_test)] = 1

print "Test accuracy: ", np.mean(y_random_class == y_test)
print "F1 score: ", metrics.f1_score(y_random_class, y_test)
print "Precision: ", metrics.precision_score(y_test, y_random_class)
print "Recall: ", metrics.recall_score(y_test, y_random_class)
print "Net value: ", my_loss_func(y_test, y_random_class)
metrics.confusion_matrix(y_test, y_random_class)

Test accuracy:  0.527
F1 score:  0.627265563436
Precision:  0.632247815727
Recall:  0.622361219703
Net value:  -650


array([[258, 463],
       [483, 796]])